In [1]:
# Imports
import torch
import torch.nn as nn
import torch.utils.data as Data
from torchvision import datasets, transforms
from torchinfo import summary
import matplotlib.pyplot as plt
import os
import numpy as np
import random
import time
import CompMethods_CSI2Image as TP
import DataSetting_v5 as DS

In [ ]:
date = '2024_Door_EXP/Comp_CSI2Image/'
name = f'{date}CSI2Image_sub_A208'
data_path = [
    '../dataset/Door_EXP/TrainVer/A208',
#             '../dataset/Door_EXP/A308',
#             '../dataset/Door_EXP/B211',
#             '../dataset/Door_EXP/C605_new'
            ]
level = 'subject'

data_organizer = DS.DataOrganizer(name, data_path, level)
data_organizer.load()

Cross validation plan at subject level
Single-process loading...
Skipping 0709A01-csio.npy
Loaded 0709A02-csi.npy of (232687, 30, 3), elapsed 10.1517 sec
Skipping 0709A02-csitime.npy
Skipping 0709A03-csitime.npy
Loaded 0709A04-pd.npy of (163332, 62), elapsed 10.2393 sec
Skipping 0709A10-csitime.npy
Loaded 0709A10-pd.npy of (93119, 62), elapsed 10.2403 sec
Loaded 0709A11-csi.npy of (219643, 30, 3), elapsed 10.2394 sec
Skipping 0709A101-csitime.npy
Loaded 0709A12-csi.npy of (151822, 30, 3), elapsed 10.2396 sec
Skipping 0709A13-csio.npy
Skipping 0709A13-csitime.npy


In [ ]:
data_organizer.load_plan(f'../dataset/Door_EXP/subject_r0.2_A208.pkl')
train_loader, valid_loader, test_loader, current_test = data_organizer.gen_loaders(
    data_organizer=data_organizer,
    subset_ratio=1,
    num_workers=2,
    csi_len=1,
    batch_size=32)

loaders = {
    'train': train_loader,
    'valid': valid_loader,
    'test': test_loader
}

In [ ]:
preprocess = TP.Preprocess(new_size=(64, 64))
gpu = 0
torch.cuda.set_device(gpu)
trainer = TP.CSI2ImageTrainer(name='CSI2Image',
                              loss_optimizer = {'LOSS': [torch.optim.Adam, 1e-4]},
                              lr=2e-4, epochs=300, cuda=gpu,
                              preprocess = preprocess,
                              notion=f'{name}_{current_test}',
                              dataloaders=loaders
                             )

trainer.schedule(early_stop=False)

In [5]:
da = {}
for p, _, file_lst in os.walk('../dataset/Door_EXP/TrainVer/A208'):
    for file_name in file_lst:
        file_name_, ext = os.path.splitext(file_name)
        if ext == '.npy' and 'rimg' in file_name_:
            da[file_name_] = np.load(os.path.join(p, file_name), mmap_mode='r')
            print(f'loaded {file_name}')

loaded 20240709_142959-env-rimg.npy
loaded 20240709_153519-rimg.npy
loaded 20240709_154950-rimg.npy
loaded 20240709_161802-rimg.npy
loaded 20240709_163045-rimg.npy
loaded 20240709_164816-rimg.npy
loaded 20240709_165643-rimg.npy
loaded 20240709_185105-rimg.npy
loaded 20240709_191754-rimg.npy
loaded 20240709_192745-rimg.npy
loaded 20240709_193258-rimg.npy
loaded 20240709_193501-rimg.npy
loaded 20240709_151047-rimg.npy


KeyboardInterrupt: 